In [11]:
import math
import pandas as pd
from operator import itemgetter

In [12]:
class DecisionTree:
    def __init__(self, df, target, positive, parent_val, parent):
        self.data = df
        self.target = target
        self.positive = positive
        self.parent_val = parent_val
        self.parent = parent
        self.childs = []
        self.decision = ''

    def _get_entropy(self, data):
        sumdata = data.shape[0]
        p = sum(data[self.target]==self.positive)
        n = sumdata - p
        p_ratio = p/sumdata
        n_ratio = n/sumdata
        if(p_ratio != 0):
            entropy_p = -p_ratio * math.log2(p_ratio)
        else:
            entropy_p = 0

        if(n_ratio != 0):
          entropy_n = -n_ratio * math.log2(n_ratio)
        else:
          entropy_n = 0

        entropy = entropy_p + entropy_n
        return entropy
    
    def _get_infogain(self, feat):
        avg_info=0
        for val in self.data[feat].unique():
            avg_info+=self._get_entropy(self.data[self.data[feat] == val])*sum(self.data[feat]==val)/self.data.shape[0]
        return self._get_entropy(df) - avg_info
    
    def _get_split(self):
        self.splitter = max(self.infogains, key = itemgetter(1))[0] 

    def generate_sub_tree(self):
        self.features = [col for col in self.data.columns if col != self.target]
        self.entropy = self._get_entropy(self.data)
        if self.entropy != 0:
            self.infogains = [(feat, self._get_infogain(feat)) for feat in self.features]
            self._get_split()
            remain_columns = [k for k in self.data.columns if k != self.splitter]
            for val in self.data[self.splitter].unique():
                df_tmp = self.data[self.data[self.splitter]==val][remain_columns]
                tmp_node = DecisionTree(df_tmp, self.target, self.positive, val, self.splitter)
                tmp_node.generate_sub_tree()
                self.childs.append(tmp_node)

In [13]:
def print_tree(n):
    for child in n.childs:
        if child:
            print(child.__dict__.get('parent', ''))
            print(child.__dict__.get('parent_val', ''), '\n')
            print_tree(child)

In [14]:
df = pd.read_csv('/content/PlayTennis.csv');
df

,Outlook,Temperature,Humidity,Wind,Play
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rainy,Mild,High,Weak,Yes
4,Rainy,Cool,Normal,Weak,Yes
5,Rainy,Cool,Normal,Strong,No
6,Overcast,Cool,Normal,Strong,Yes
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes
9,Rainy,Mild,Normal,Weak,Yes


In [15]:
dt = DecisionTree(df, 'Play', 'Yes', '', '')
dt.generate_sub_tree()
print_tree(dt)

Outlook
Sunny 

Humidity
High 

Humidity
Normal 

Outlook
Overcast 

Outlook
Rainy 

Wind
Weak 

Wind
Strong 

